In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
import sys

sys.path.append('../')  # 返回notebook的上一级目录
# sys.path.append('E:\GitHub\QA-abstract-and-reasoning')  # 效果同上

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from utils.loader import *
from utils.config import *

from gensim.models.word2vec import LineSentence, Word2Vec
import tensorflow as tf
# from model_layer import seq2seq_model
import time

# 加载数据

In [3]:
train_X,train_Y,test_X = load_dataset()

In [29]:
vocab_index,index_vocab = load_vocab(VOCAB_INDEX_PAD)

In [ ]:
embedding_matrix = np.loadtxt(EMBEDDING_MATRIX_PAD)